In [33]:
from goesaws import *
import datetime
import pandas as pd
import sys

date = str(datetime.date.today())

bucket_name = 'noaa-goes16'
product_name = 'ABI-L2-MCMIPM'
lightning_mapper = 'GLM-L2-LCFA'
yr = 2022
day_of_year =141
hr = 18
minutes = 1

In [34]:
abiprefix = gen_prefix(product=product_name,year = yr, day=day_of_year, hour = hr)
abifiles = gen_fn(bucket=bucket_name,prefix=abiprefix)
abidatasets = [gen_data(key=abifiles[i], bucket = bucket_name) for i in range(0,minutes)]
abiDS = xr.concat(abidatasets,dim = 't')

In [35]:
skyprefix = gen_prefix(product = "ABI-L2-ACMM", year = yr, day = day_of_year, hour = hr)
skyfiles = gen_fn(bucket = bucket_name, prefix=skyprefix)
skydatasets = [gen_data(key=skyfiles[i], bucket = bucket_name) for i in range(0,minutes)]
skyDS = xr.concat(skydatasets,dim = 't')

In [36]:
dataset2_resampled = skyDS.interp(
    x=abiDS["x"].data,
    y=abiDS["y"].data,
    t = abiDS["t"].data,
    method='linear',
    kwargs={'fill_value': "extrapolate"}
)

/Users/robbiefeldstein/anaconda3/envs/GOES/lib/python3.11/site-packages/scipy/interpolate/_interpolate.py:701: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]


In [37]:
nfile = xr.merge([abiDS,dataset2_resampled])
nfile = nfile.coarsen(x=4,y=4).mean()
nfile = calc_latlon(nfile)

In [38]:
df = nfile.to_dataframe()
df

CMI_C01  \
t                             y        x         number_of_time_bounds number_of_image_bounds RTM_BT_comparison_bands number_of_focal_planes_used number_of_LZA_bounds number_of_SZA_bounds band             
2022-05-21 18:00:31.272519936 0.108528 -0.040768 0                     0                      0                       0                           0                    0                    0     0.481706   
                                                                                                                                                                       1                    0     0.481706   
                                                                                                                                                  1                    0                    0     0.481706   
                                                                                                                                                                       1                    0     0.481706   
                                                                                                                      1                           0                    0                    0     0.481706   
...                                                                                                                                                                                                    ...   
                              0.080752 -0.012992 1                     1                      1                       0                           1                    1                    0     0.155694   
                                                                                                                      1                           0                    0                    0     0.155694   
                                                                                                                                                                       1                    0     0.155694   
                                                                                                                                                  1                    0                    0     0.155694   
                                                                                                                                                                       1                    0     0.155694   

                                                                                                                                                                                                  DQF_C01  \
t                             y        x         number_of_time_bounds number_of_image_bounds RTM_BT_comparison_bands number_of_focal_planes_used number_of_LZA_bounds number_of_SZA_bounds band            
2022-05-21 18:00:31.272519936 0.108528 -0.040768 0                     0                      0                       0                           0                    0                    0         0.0   
                                                                                                                                                                       1                    0         0.0   
                                                                                                                                                  1                    0                    0         0.0   
                                                                                                                                                                       1                    0         0.0   
                                                                                                                      1                           0                    0                    0         0.0   
...                                                                              

In [39]:
features = ["CMI_C01", "CMI_C02","CMI_C03","CMI_C04","CMI_C05","CMI_C06","CMI_C07","CMI_C08","CMI_C09","CMI_C10","CMI_C11","CMI_C12","CMI_C13","CMI_C14","CMI_C15","CMI_C16","ACM", "BCM", "Cloud_Probabilities","lat","lon"]
ndf = df[features].copy()
ndf["time"] = ndf.index.get_level_values('t')
ndf.drop_duplicates(inplace=True)
ndf

CMI_C01  \
t                             y        x         number_of_time_bounds number_of_image_bounds RTM_BT_comparison_bands number_of_focal_planes_used number_of_LZA_bounds number_of_SZA_bounds band             
2022-05-21 18:00:31.272519936 0.108528 -0.040768 0                     0                      0                       0                           0                    0                    0     0.481706   
                                       -0.040544 0                     0                      0                       0                           0                    0                    0     0.365158   
                                       -0.040320 0                     0                      0                       0                           0                    0                    0     0.318968   
                                       -0.040096 0                     0                      0                       0                           0                    0                    0     0.310932   
                                       -0.039872 0                     0                      0                       0                           0                    0                    0     0.373254   
...                                                                                                                                                                                                    ...   
                              0.080752 -0.013888 0                     0                      0                       0                           0                    0                    0     0.308531   
                                       -0.013664 0                     0                      0                       0                           0                    0                    0     0.226706   
                                       -0.013440 0                     0                      0                       0                           0                    0                    0     0.198750   
                                       -0.013216 0                     0                      0                       0                           0                    0                    0     0.150635   
                                       -0.012992 0                     0                      0                       0                           0                    0                    0     0.155694   

                                                                                                                                                                                                   CMI_C02  \
t                             y        x         number_of_time_bounds number_of_image_bounds RTM_BT_comparison_bands number_of_focal_planes_used number_of_LZA_bounds number_of_SZA_bounds band             
2022-05-21 18:00:31.272519936 0.108528 -0.040768 0                     0                      0                       0                           0                    0                    0     0.431289   
                                       -0.040544 0                     0                      0                       0                           0                    0                    0     0.305932   
                                       -0.040320 0                     0                      0                       0                           0                    0                    0     0.255297   
                                       -0.040096 0                     0                      0                       0                           0                    0                    0     0.245516   
                                       -0.039872 0                     0                      0                       0                           0                    0                    0     0.310456   
...                                                                       

In [40]:
glmprefix = gen_prefix(product = "GLM-L2-LCFA", year = yr, day = day_of_year, hour = hr)
glmfiles = gen_fn(bucket = bucket_name, prefix=glmprefix)
#There should be 180 GLM files per hour (one every 20 seconds)
glmdatasets = [gen_data(key=glmfiles[i], bucket = bucket_name) for i in range(0,minutes*3)]
latitudes = [np.array(glmdatasets[i]["event_lat"].data) for i in range(0,minutes*3)]
lats = np.concatenate(latitudes)
longitudes = [np.array(glmdatasets[i]["event_lon"].data) for i in range(0,minutes*3)]
lons= np.concatenate(longitudes)


strikes = list(zip(lats,lons))

In [41]:
from scipy.spatial import cKDTree

ndf = ndf.reset_index(drop=True)  # reset the index of the ndf dataframe
tree = cKDTree(ndf[['lat', 'lon']].values)
distances, indices = tree.query(strikes)

# filter out the indices that are not present in the ndf dataframe
valid_indices = indices[indices < len(ndf)]

lightning_df = pd.DataFrame({
    'strike_lat': [strike[0] for strike in strikes],
    'strike_lon': [strike[1] for strike in strikes],
    'nearest_lat': ndf.loc[valid_indices, 'lat'].values,
    'nearest_lon': ndf.loc[valid_indices, 'lon'].values,
    'distance': distances[indices < len(ndf)]
})


lightning_df["lightning"] = 0
lightning_df.loc[distances < 5, 'lightning'] = 1
lightning_df["Coordinates"] = list(zip(lightning_df["nearest_lat"],lightning_df["nearest_lon"]))
#len(lightning_df)



In [42]:
features = ["CMI_C01", "CMI_C02","CMI_C03","CMI_C04","CMI_C05","CMI_C06","CMI_C07","CMI_C08","CMI_C09","CMI_C10","CMI_C11","CMI_C12","CMI_C13","CMI_C14","CMI_C15","CMI_C16","ACM", "BCM", "Cloud_Probabilities","lat","lon","Coordinates","time","Lightning"]
ndf["Coordinates"] = list(zip(ndf["lat"],ndf["lon"]))

ndf



,CMI_C01,CMI_C02,CMI_C03,CMI_C04,CMI_C05,CMI_C06,CMI_C07,CMI_C08,CMI_C09,CMI_C10,...,CMI_C14,CMI_C15,CMI_C16,ACM,BCM,Cloud_Probabilities,lat,lon,time,Coordinates
0,0.481706,0.431289,0.579424,0.002143,0.478710,0.376825,302.558289,239.462845,247.805496,255.412079,...,275.379395,274.171387,263.115356,NaN,NaN,NaN,40.131033,-93.381551,2022-05-21 18:00:31.272519936,"(40.13103323474366, -93.38155072424266)"
1,0.365158,0.305932,0.495337,0.002024,0.376627,0.285793,300.493988,239.296509,247.773743,255.493149,...,276.071411,274.726074,263.449341,NaN,NaN,NaN,40.126986,-93.274144,2022-05-21 18:00:31.272519936,"(40.12698592712501, -93.2741436595977)"
2,0.318968,0.255297,0.471785,0.003135,0.333135,0.245873,299.167969,238.837036,247.442963,255.309204,...,276.191132,274.573425,263.160126,NaN,NaN,NaN,40.122966,-93.166853,2022-05-21 18:00:31.272519936,"(40.122965705716815, -93.16685328500078)"
3,0.310932,0.245516,0.473551,0.006746,0.323313,0.240774,298.233246,237.817749,246.410950,254.217865,...,274.687378,272.764282,261.356201,NaN,NaN,NaN,40.118972,-93.059679,2022-05-21 18:00:31.272519936,"(40.11897247748837, -93.05967862352706)"
4,0.373254,0.310456,0.511190,0.020992,0.365952,0.287559,295.210480,235.055664,243.140259,250.491760,...,267.707275,265.780762,255.517563,NaN,NaN,NaN,40.115006,-92.952619,2022-05-21 18:00:31.272519936,"(40.115006150366405, -92.95261870531907)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15620,0.308531,0.242341,0.260357,0.044682,0.222599,0.208353,293.009521,230.690750,238.557037,245.590149,...,261.052612,257.352905,249.857941,NaN,NaN,NaN,27.802960,-80.174225,2022-05-21 18:00:31.272519936,"(27.80295961693488, -80.17422458142033)"
15621,0.226706,0.156210,0.159305,0.029861,0.135099,0.127282,294.775055,231.126450,239.297974,247.348755,...,269.446686,265.051147,255.052811,NaN,NaN,NaN,27.802322,-80.090399,2022-05-21 18:00:31.272519936,"(27.802322483769377, -80.09039916473711)"
15622,0.198750,0.126250,0.123214,0.020278,0.102381,0.090655,296.940002,232.209106,240.846008,249.637421,...,276.916840,272.343628,260.316528,NaN,NaN,NaN,27.801696,-80.006592,2022-05-21 18:00:31.272519936,"(27.801695861529403, -80.0065919270383)"
15623,0.150635,0.074861,0.063234,0.006746,0.046429,0.036567,298.697327,232.964310,242.203491,252.384430,...,288.052795,283.820251,268.668274,NaN,NaN,NaN,27.801080,-79.922803,2022-05-21 18:00:31.272519936,"(27.801079742704182, -79.92280256024858)"


In [43]:
lightning_df.drop_duplicates(inplace=True)
lightning_df


,strike_lat,strike_lon,nearest_lat,nearest_lon,distance,lightning,Coordinates
0,35.918076,-91.139566,35.958094,-91.091811,0.062306,1,"(35.9580943396289, -91.09181131591522)"
1,35.909951,-90.942532,35.853800,-90.968953,0.062056,1,"(35.85380036926935, -90.96895315061255)"
2,35.993234,-90.960814,35.955151,-90.993307,0.050061,1,"(35.955151310266054, -90.99330708309435)"
3,36.074485,-90.981127,36.056706,-91.017835,0.040788,1,"(36.05670583891687, -91.01783539835132)"
4,36.157767,-91.001439,36.158466,-91.042540,0.041107,1,"(36.158465544384924, -91.04253992200952)"
...,...,...,...,...,...,...,...
18791,35.548384,-82.979915,35.575807,-83.026106,0.053718,1,"(35.57580723385387, -83.02610571694247)"
18877,34.644464,-81.318328,34.662710,-81.363588,0.048800,1,"(34.66271016250764, -81.3635878462917)"
18886,30.884565,-82.583815,30.880935,-82.625580,0.041922,1,"(30.88093547453507, -82.62557997979242)"
18951,33.854296,-81.159888,33.881324,-81.204769,0.052391,1,"(33.881324287169384, -81.20476861399247)"


In [44]:

strike_df = lightning_df[lightning_df["lightning"] == 1]
#Just Coords Represents lightning strikes and relative strikes in order of time 
just_coords = pd.DataFrame(strike_df["Coordinates"].value_counts())
just_coords["Counts"] = just_coords["Coordinates"]
just_coords["Coordinates"] = just_coords.index
just_coords.reset_index(inplace=True,drop=True)
just_coords


,Coordinates,Counts
0,"(27.85446632383459, -84.82189637996795)",21
1,"(34.162308721554766, -80.2312646837825)",20
2,"(27.853235498171262, -84.73657767177431)",20
3,"(27.850807249908655, -84.56604849182719)",19
4,"(36.9153395409372, -92.42730243983118)",16
...,...,...
934,"(38.33738004063121, -87.68807003268911)",1
935,"(38.339917457790335, -87.7878004324142)",1
936,"(38.23644436579997, -87.86558500916998)",1
937,"(31.35538105737299, -88.64036849722561)",1


In [45]:
ndf["Coordinates"].value_counts()

(40.13103323474366, -93.38155072424266)     1
(31.625469031219996, -88.23367102678168)    1
(31.62150937750704, -88.0518932063424)      1
(31.61955149661157, -87.96108875848343)     1
(31.617608207220044, -87.87033999200665)    1
                                           ..
(35.586042058100155, -83.68016321132893)    1
(35.58452942190898, -83.5866098115931)      1
(35.583033663620164, -83.49309606398775)    1
(35.58155475988619, -83.3996214987261)      1
(27.800474119912803, -79.83903075680016)    1
Name: Coordinates, Length: 15625, dtype: int64

In [46]:
#isin method does not take into consideration the time element of the lightning data
ndf["Lightning"] = ndf["Coordinates"].isin(lightning_df["Coordinates"])
nmap = {True:1, False:0}
ndf["Lightning"] = ndf["Lightning"].map(nmap)
subset = ndf[features]
subset
    

,CMI_C01,CMI_C02,CMI_C03,CMI_C04,CMI_C05,CMI_C06,CMI_C07,CMI_C08,CMI_C09,CMI_C10,...,CMI_C15,CMI_C16,ACM,BCM,Cloud_Probabilities,lat,lon,Coordinates,time,Lightning
0,0.481706,0.431289,0.579424,0.002143,0.478710,0.376825,302.558289,239.462845,247.805496,255.412079,...,274.171387,263.115356,NaN,NaN,NaN,40.131033,-93.381551,"(40.13103323474366, -93.38155072424266)",2022-05-21 18:00:31.272519936,1
1,0.365158,0.305932,0.495337,0.002024,0.376627,0.285793,300.493988,239.296509,247.773743,255.493149,...,274.726074,263.449341,NaN,NaN,NaN,40.126986,-93.274144,"(40.12698592712501, -93.2741436595977)",2022-05-21 18:00:31.272519936,0
2,0.318968,0.255297,0.471785,0.003135,0.333135,0.245873,299.167969,238.837036,247.442963,255.309204,...,274.573425,263.160126,NaN,NaN,NaN,40.122966,-93.166853,"(40.122965705716815, -93.16685328500078)",2022-05-21 18:00:31.272519936,0
3,0.310932,0.245516,0.473551,0.006746,0.323313,0.240774,298.233246,237.817749,246.410950,254.217865,...,272.764282,261.356201,NaN,NaN,NaN,40.118972,-93.059679,"(40.11897247748837, -93.05967862352706)",2022-05-21 18:00:31.272519936,0
4,0.373254,0.310456,0.511190,0.020992,0.365952,0.287559,295.210480,235.055664,243.140259,250.491760,...,265.780762,255.517563,NaN,NaN,NaN,40.115006,-92.952619,"(40.115006150366405, -92.95261870531907)",2022-05-21 18:00:31.272519936,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15620,0.308531,0.242341,0.260357,0.044682,0.222599,0.208353,293.009521,230.690750,238.557037,245.590149,...,257.352905,249.857941,NaN,NaN,NaN,27.802960,-80.174225,"(27.80295961693488, -80.17422458142033)",2022-05-21 18:00:31.272519936,1
15621,0.226706,0.156210,0.159305,0.029861,0.135099,0.127282,294.775055,231.126450,239.297974,247.348755,...,265.051147,255.052811,NaN,NaN,NaN,27.802322,-80.090399,"(27.802322483769377, -80.09039916473711)",2022-05-21 18:00:31.272519936,0
15622,0.198750,0.126250,0.123214,0.020278,0.102381,0.090655,296.940002,232.209106,240.846008,249.637421,...,272.343628,260.316528,NaN,NaN,NaN,27.801696,-80.006592,"(27.801695861529403, -80.0065919270383)",2022-05-21 18:00:31.272519936,0
15623,0.150635,0.074861,0.063234,0.006746,0.046429,0.036567,298.697327,232.964310,242.203491,252.384430,...,283.820251,268.668274,NaN,NaN,NaN,27.801080,-79.922803,"(27.801079742704182, -79.92280256024858)",2022-05-21 18:00:31.272519936,0


In [47]:
subset["Lightning"].value_counts()

0    14637
1      988
Name: Lightning, dtype: int64

Right now, all of the lightning strikes might not necessarily be close to the assigned value. Also we have to deal with the time component. But this is much better than before. 

In [48]:
#subset.to_csv("/Users/robbiefeldstein/Documents/Programming/Research/Datasets/May_22.csv")
